In [6]:
!pip install --upgrade sagemaker datasets chess

  Using cached chess-1.11.1-py3-none-any.whl


In [7]:
# restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

In [5]:
import boto3
import sagemaker
import transformers
print(f"Sagemaker Version - {sagemaker.__version__}")
print(f"Transformers version - {transformers.__version__}")

sess = sagemaker.Session()
# sagemaker session bucket -> used for uploading data, models and logs
# sagemaker will automatically create this bucket if it not exists
sagemaker_session_bucket=None
if sagemaker_session_bucket is None and sess is not None:
    # set to default bucket if a bucket name is not given
    sagemaker_session_bucket = sess.default_bucket()
 
try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']
 
sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)
 
print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
Sagemaker Version - 2.232.2
Transformers version - 4.38.2
sagemaker role arn: arn:aws:iam::769977401909:role/service-role/SageMaker-ExecutionRole-20231218T134780
sagemaker bucket: sagemaker-us-east-1-769977401909
sagemaker session region: us-east-1


In [91]:
model_id, model_version = "meta-textgeneration-llama-3-1-8b", "2.2.2"
train_test_data_location = f's3://{sess.default_bucket()}/datasets/chess-world-fide-championship'
local_train_data_file = "data/train.jsonl"
local_test_data_file = "data/validation.jsonl"

In [92]:
import time
def interactive_sleep(seconds: int):
    dots = ''
    for i in range(seconds):
        dots += '.'
        print(dots, end='\r')
        time.sleep(1)

In [93]:
from sagemaker.jumpstart.model import JumpStartModel
pretrained_model = JumpStartModel(model_id=model_id, model_version=model_version)

No instance type selected for inference hosting endpoint. Defaulting to ml.g5.4xlarge.
INFO:sagemaker.jumpstart:No instance type selected for inference hosting endpoint. Defaulting to ml.g5.4xlarge.


In [105]:
import chess.pgn
from pathlib import Path
import json

instruction = '''In the FEN Lowercase letters describe the black pieces. "p" stands for pawn, "r" for rook, "n" for knight, "b" for bishop, "q" for queen, and "k" for king.
The same letters are used for the white pieces, but they appear in uppercase.
Empty squares are denoted by numbers from one to eight, depending on how many empty squares are between two pieces.
Use the FEN to understand the position of the pieces on the chessboard and recommend legal moves accordingly and follow the rules of playing chess to recommend legal moves.'''

pathlist = Path("data/pgn/").glob('**/*.pgn')
s = ","
with open("data/pgn/data.json", 'w') as f:
    for path in pathlist:
        print(f'File being processed - {path}')
        pgn = open(path)
        while True:
            game = chess.pgn.read_game(pgn)
            if game is None:
                break
            else:
                event = game.headers["Event"]
                white_player = game.headers["White"]
                black_player = game.headers["Black"]
                board = game.board()
                prev_moves = []
                move_cnt = 0
                for move in game.mainline_moves():
                    move_cnt += 1
                    next_move_color= "WHITE" if board.turn else "BLACK"
                    move_json = {
                        "instruction": instruction,
                        "context": f'''You are a chess grandmaster. You are playing {next_move_color} color and the current chessboard FEN is {board.fen()}. ''',
                        "response":move.uci()
                    }
                    prev_moves.append(f"{move.uci()}")
                    board.push(move)
                    f.write(json.dumps(move_json) + "\n")

from datasets import load_dataset
dataset = load_dataset("json", data_files=f"{path.cwd()}/data/pgn/data.json", split="train[:100%]")
dataset = dataset.train_test_split(test_size=0.3)
print(f'Schema for dataset: {dataset}')

dataset = dataset.shuffle()
columns_to_remove = list(dataset["train"].features)

dataset["train"].to_json(local_train_data_file, orient="records", force_ascii=False)
dataset["test"].to_json(local_test_data_file, orient="records", force_ascii=False)

File being processed - data/pgn/FideChamp1999.pgn
File being processed - data/pgn/FideChamp2002.pgn
File being processed - data/pgn/WorldChamp2006.pgn
File being processed - data/pgn/WorldChamp2013.pgn
File being processed - data/pgn/WorldChamp2004.pgn
File being processed - data/pgn/FideChamp2004.pgn
File being processed - data/pgn/WorldChamp2000.pgn
File being processed - data/pgn/FideChamp2000.pgn
File being processed - data/pgn/WorldChamp2008.pgn
File being processed - data/pgn/WorldChamp2012.pgn
File being processed - data/pgn/WorldChamp2021.pgn
File being processed - data/pgn/FideChamp1996.pgn
File being processed - data/pgn/WorldChamp2016.pgn
File being processed - data/pgn/WorldChamp2007.pgn
File being processed - data/pgn/WorldChamp2014.pgn
File being processed - data/pgn/WorldChamp2023.pgn
File being processed - data/pgn/FideChamp2005.pgn


ERROR:chess.pgn:illegal san: 'Bf4' in 8/8/R1p5/1p1k4/1Pr5/1K6/8/8 w - - 18 115 while parsing <Game at 0x7f7d4ddade90 ('Caruana, Fabiano' vs. 'Carlsen, Magnus', '2018.11.09' at 'London ENG')>
ERROR:chess.pgn:illegal san: 'd4' in 8/8/R1p5/1p1k4/1Pr5/1K6/8/8 w - - 18 115 while parsing <Game at 0x7f7d4ddade90 ('Caruana, Fabiano' vs. 'Carlsen, Magnus', '2018.11.09' at 'London ENG', 1 errors)>


File being processed - data/pgn/WorldChamp2010.pgn
File being processed - data/pgn/WorldChamp2018.pgn


Generating train split: 0 examples [00:00, ? examples/s]

Schema for dataset: DatasetDict({
    train: Dataset({
        features: ['instruction', 'context', 'response'],
        num_rows: 105373
    })
    test: Dataset({
        features: ['instruction', 'context', 'response'],
        num_rows: 45161
    })
})


Creating json from Arrow format:   0%|          | 0/106 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/46 [00:00<?, ?ba/s]

33623397

In [106]:
import json

template = {
    "prompt": (
        "What is your next move in UCI notation. "
        "Provide only the UCI notation for your next move. Your next move should be a valid legal move only for the color you are playing.\n\n"
        "### Instruction:\n{instruction}\n\n### Input:\n{context}\n\n"
    ),
    "completion": " {response}",
}

with open("data/template.json", "w") as f:
    json.dump(template, f)

In [107]:
from sagemaker.s3 import S3Uploader
import sagemaker
import random

S3Uploader.upload(local_train_data_file, train_test_data_location)
S3Uploader.upload(local_test_data_file, train_test_data_location)
S3Uploader.upload("data/template.json", train_test_data_location)

print(f"Training data: {train_test_data_location}")

Training data: s3://sagemaker-us-east-1-769977401909/datasets/chess-world-fide-championship


In [ ]:
from sagemaker.jumpstart.estimator import JumpStartEstimator

estimator = JumpStartEstimator(
    model_id=model_id,
    model_version=model_version,
    environment={"accept_eula": "true"},  # Please change this to {"accept_eula": "true"}
    disable_output_compression=True,
    instance_type="ml.g5.24xlarge"
)
# By default, instruction tuning is set to false. Thus, to use instruction tuning dataset you use
estimator.set_hyperparameters(instruction_tuned=True, epoch="3", max_input_length="1024")
estimator.fit({"training": train_test_data_location})

INFO:sagemaker:Creating training-job with name: meta-textgeneration-llama-3-1-8b-2024-10-23-23-03-38-240


2024-10-23 23:03:40 Starting - Starting the training job
2024-10-23 23:03:40 Pending - Training job waiting for capacity......
2024-10-23 23:04:33 Pending - Preparing the instances for training......
2024-10-23 23:05:17 Downloading - Downloading input data..............................
2024-10-23 23:10:35 Training - Training image download completed. Training in progress..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2024-10-23 23:10:37,977 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2024-10-23 23:10:38,014 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2024-10-23 23:10:38,024 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2024-10-23 23:10:38,026 sagemaker_pytorch_container.training INFO     Invoking user training script.
2024-10-23 23:10:47,484 sagemaker-training-toolkit INFO     Installing 

In [112]:
estimator.model_data

{'S3DataSource': {'S3Uri': 's3://sagemaker-us-east-1-769977401909/meta-textgeneration-llama-3-1-8b-2024-10-23-23-03-38-240/output/model/',
  'S3DataType': 'S3Prefix',
  'CompressionType': 'None'}}

In [114]:
rivchess_model_src = {"s3DataSource": {"s3Uri": f'{ estimator.model_data["S3DataSource"]["S3Uri"] }'}}
%store rivchess_model_src

Stored 'rivchess_model_src' (dict)


### Import the fine tuned model into Bedrock as Custom Model

In [115]:
import boto3
import datetime
import boto3
import json
from botocore.exceptions import ClientError
print(boto3.__version__)

1.35.46


In [117]:
br_client = boto3.client('bedrock', region_name='us-east-1')
br_run_client = boto3.client('bedrock-runtime', region_name='us-east-1')

model_split = model_id.split("/")
if len(model_split) > 1:
    rivchess_model_nm = f"RIVCHESS-{model_split[1]}"
else:
    rivchess_model_nm = f"RIVCHESS-{model_split[0]}"

rivchess_imp_jb_nm = f"{rivchess_model_nm}-job-{datetime.datetime.now().strftime('%Y%m%d%M%H%S')}"
role_arn = role

create_model_import_job_resp = br_client.create_model_import_job(jobName=rivchess_imp_jb_nm,
                                  importedModelName=rivchess_model_nm,
                                  roleArn=role_arn,
                                  modelDataSource=rivchess_model_src)

In [118]:
list_model_import_jobs_response = br_client.list_model_import_jobs(
    nameContains=rivchess_imp_jb_nm)

print(f"BR CMI Import Job - {create_model_import_job_resp['jobArn']} is - {list_model_import_jobs_response['modelImportJobSummaries'][0]['status']}")
while list_model_import_jobs_response['modelImportJobSummaries'][0]['status'] != 'Completed':
    interactive_sleep(30)
    list_model_import_jobs_response = br_client.list_model_import_jobs(nameContains=rivchess_imp_jb_nm)

BR CMI Import Job - arn:aws:bedrock:us-east-1:769977401909:model-import-job/7iab154rn07o is - InProgress


### Invoke the imported model using Bedrock API's

In [119]:
try:
    get_imported_model_response = br_client.get_imported_model(
        modelIdentifier=rivchess_model_nm
    )

    br_model_id = get_imported_model_response['modelArn']
    br_model_id
except br_client.exceptions.ResourceNotFoundException:
    print("Model not yet imported")

In [120]:
def call_invoke_model_and_print(native_request):
    request = json.dumps(native_request)

    try:
        # Invoke the model with the request.
        response = br_run_client.invoke_model(modelId=br_model_id, body=request)
        model_response = json.loads(response["body"].read())
        # print(f"model_response: {model_response}")
        response_text = model_response['generation'].replace("\n", "").replace("### Response:", "")
        return response_text
    except (ClientError, Exception) as e:
        print(f"ERROR: Can't invoke '{br_model_id}'. Reason: {e}")
        exit(1)

In [123]:
template["prompt"]

'What is your next move in UCI notation. Provide only the UCI notation for your next move. Your next move should be a valid legal move only for the color you are playing.\n\n### Instruction:\n{instruction}\n\n### Input:\n{context}\n\n'

In [126]:
instruction = '''In the FEN Lowercase letters describe the black pieces. "p" stands for pawn, "r" for rook, "n" for knight, "b" for bishop, "q" for queen, and "k" for king.
The same letters are used for the white pieces, but they appear in uppercase.
Empty squares are denoted by numbers from one to eight, depending on how many empty squares are between two pieces.
Use the FEN to understand the position of the pieces on the chessboard and recommend legal moves accordingly and follow the rules of playing chess to recommend legal moves.'''

move_color = "BLACK"
board_fen = "6k1/p1q1prbp/b3n1p1/2pPPp2/5P1Q/4BN2/Pr2N1PP/R1R4K b - - 0 21"

context = f"You are a chess grandmaster. You are playing {move_color} color and the current chessboard FEN is {board_fen}."

formatted_prompt = template["prompt"].format(instruction=instruction, 
                          context=context,
                          answer="")

native_request = {
    "prompt": formatted_prompt,
    "max_tokens": 50,
    "top_p": 0.9,
    "temperature": 0.1
}

call_invoke_model_and_print(native_request)

' e6f4'

In [127]:
move_color = "WHITE"
board_fen = "1r1qk2r/p2bppbp/n2P1np1/1pp5/3P4/2P1BB2/PP2QPPP/RN2K1NR w KQk - 3 11"

formatted_prompt = template["prompt"].format(instruction=instruction, 
                          context=context,
                          answer="")

native_request = {
    "prompt": formatted_prompt,
    "max_tokens": 100,
    "top_p": 0.9,
    "temperature": 0.1,
    "top_k": 50,
}

call_invoke_model_and_print(native_request)

' a2a4'

### Clean Up

In [116]:
delete_imported_model_response = br_client.delete_imported_model(
    modelIdentifier=br_model_id
)